# Model-Data Comparison Workflow using Kamodo and Pysat
This workflow uses Kamodo to compare simulated in-situ values with observational values along a satellite trajectory. 
### Calculating the simulated values along a trajectory
The following block retrieves a real satellite trajectory from SSCWeb and calculates the simulated values of the requested variable(s) for each trajectory location. The code snippet below works for a variety of model names, satellites, time ranges, variables, and coordinate systems, so feel free to play. The output of the RealFlight function is functionalized with a single line at the bottom of the block. The output of kamodo_object shows the LaTeX representation of the variable(s) requested in the RealFlight function call.

In [ ]:
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
import datetime as dt

# Set input values for RealFlight function call.
model = 'TIEGCM'
file_dir = 'C:/Users/rringuet/Kamodo_Data/TIEGCM/Uriel_Ramirez_012517_IT_1/'
dataset = 'cnofs'
start_utcts = dt.datetime(2015, 3, 18, 0).replace(tzinfo=dt.timezone.utc).timestamp()
end_utcts = dt.datetime(2015, 3, 21, 0).replace(tzinfo=dt.timezone.utc).timestamp()-1
variable_list = ['T_i'] # list of desired variable names
coord_type = 'GEO'  # GEO cartesian coordinates for the trajectory.
results = SF.RealFlight(dataset, start_utcts, end_utcts, model, file_dir, 
                        variable_list, coord_type)

# Functionalize the results
kamodo_object = SF.O.Functionalize_SFResults(model,results)
kamodo_object

### Downloading and cleaning the observational data
The following code block uses pysat to download and clean the observational data. The satellite is chosen to be the same satellite as in the above block so the same trajectory is used. The instrument is chosen to correspond to the same variable as in the simulated data above for a proper comparison. Please feel free to change the satellite and variable names in parallel with any changes in the above code block. Keep in mind that different satellites are stored in different sections of pysat, which may require installing additional portions of pysat.

In [ ]:
# Get CINDI ion temperature data using pysat.
import pysat  # pip install pysat
import pysatNASA  # pip install pysatNASA

pysat_datadir = 'C:/Users/rringuet/Kamodo_Data/pysat_data/'
# pysat.params['data_dirs'] = pysat_datadir  # Only needed the first time.
pysat.utils.registry.register(['pysatNASA.instruments.cnofs_ivm'])
ivm = pysat.Instrument('cnofs', 'ivm', inst_id='', clean_level='clean')
start_download_date = dt.datetime.utcfromtimestamp(start_utcts)
stop_download_date = dt.datetime.utcfromtimestamp(end_utcts)
# ivm.download(start_download_date, stop_download_date)  # already on machine
ivm.load(date=start_download_date, end_date=stop_download_date)

### Functionalizing the observational data
Next, we must functionalize the observational data to access the capabilities of Kamodo for the comparison. Pysat returns data in pandas dataframes, so a few lines of code are necessary to retrieve the utc timestamps. Once that is completed, the observational time series is functionalized in a single command (kamodo_object = ...). With both the simulated and observational data in the same Kamodo object, various Kamodo capabilities can be used to analyze the datasets, including function composition as shown at the end of the block. If a different measurement is used in the previous blocks, you will need to use pysat's features to retrieve the proper unit and variable name for the observational data.

In [ ]:
# Convert pandas timestamps into utc timestamps
import numpy as np
utc_time = ivm.index.values.astype(np.int64)/1e9

# Add to kamodo object.
kamodo_object = SF.O.Functionalize_TimeSeries(utc_time, 'CINDIT_i', 'K', 
                                               ivm.data['ionTemperature'].values, 
                                               kamodo_object)
kamodo_object['DIFF'] = 'abs(CINDIT_i - T_i)'
kamodo_object

### Visualization
Now that the simulated and observational data have been functionalized, we can easily plot any functions desired with a single command. The figure can be saved in full interactive form. Please see the Plotly documentation for more details on this feature. https://plotly.com/python-api-reference/index.html

In [ ]:
# Now we can plot model and data on the same figure with the difference
kamodo_object.plot('T_i','CINDIT_i','DIFF')